In [1]:
# general
from datetime import datetime
from dotenv import load_dotenv, find_dotenv
import os

# custom local libs
from function import baseball_lambda

/Users/rosclay/Documents/VSCode/langgraph-multi-agent/venv/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `bind_tools` is in beta. It is actively being worked on, so the API may change.
  warn_beta(
Python REPL can execute arbitrary code. Use with caution.


In [2]:
# read local .env file
_ = load_dotenv(find_dotenv()) 

In [3]:
# set Langsmith project
today = datetime.now().strftime("%Y%m%d")
os.environ["LANGCHAIN_PROJECT"] = f"Baseball Curveballs - {today}"

In [4]:
session_id = '2110956'

In [5]:
task =  "Consider the first week of August 2020 - find the names of 3 pitchers who's curveballs were most similar to Max Scherzer's."
result = baseball_lambda.execute_workflow(task, session_id)
print(result)


*** Entered Orchestrate Node ***

*** Entered Retrieve Node ***

Distance to neareast plan: 0.6246547698974609

*** Entered Initialize Node ***

Formulating a new plan to solve the task.

*** Entered Update Node ***

Updating plan based function documentation
1. Use playerid_lookup to find Max Scherzer's MLBAM player ID 
2. Use statcast_pitcher with Scherzer's player ID to get his curveball data for the first week of August 2020
3. Calculate the average velocity, spin rate, and movement for Scherzer's curveballs 
4. Use statcast to get pitching data for all other pitchers in that same time period
5. Filter the results to only curveballs
6. For each pitcher, calculate the average velocity, spin rate, and movement of their curveballs
7. Calculate the difference between each pitcher's averages and Scherzer's averages
8. Sum the absolute differences to get a "similarity score" - the lower the score, the more similar to Scherzer
9. Find the 3 pitchers with the lowest similarity scores 
10.

In [6]:
task = """
make sure the plan follows this general flow:
1) create an average vector for each pitcher's curveball.
2) Train a knn model on this data
3) Use this model to find the 3 pitchers with the most similar curveball to Max Scherzer
"""
result = baseball_lambda.execute_workflow(task, session_id)
print(result)


*** Entered Orchestrate Node ***

*** Entered Revise Node ***

The revised plan to find the 3 pitchers most similar to Max Scherzer's curveball in the first week of August 2020 is as follows:

1. Use playerid_lookup to find Max Scherzer's MLBAM player ID.

2. Use statcast_pitcher with the following parameters to get Scherzer's curveball data for the first week of August 2020:
   - start_dt = '2020-08-01'
   - end_dt = '2020-08-07'
   - player_id = Max Scherzer's MLBAM ID

3. From the returned data, calculate the average release_speed, release_spin_rate, pfx_x, and pfx_z for Scherzer's curveballs. Store these 4 values in a "Scherzer curveball average vector".

4. Use statcast with the following parameters to get pitching data for all pitchers in the first week of August 2020:
   - start_dt = '2020-08-01'
   - end_dt = '2020-08-07'

5. Filter the results to only pitches where pitch_type is 'CU' for curveball.

6. Group the data by pitcher (using their MLBAM ID). For each pitcher, calcul

In [7]:
task = 'approved'
result = baseball_lambda.execute_workflow(task, session_id)
print(result)


*** Entered Orchestrate Node ***
Unknown plan. Executing with Opus

*** Entered Generate Node ***


*** Entered Execute Node ***


Executing:
from pybaseball import playerid_lookup, statcast_pitcher, statcast
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

# Step 1: Find Max Scherzer's player ID
scherzer_id = playerid_lookup('scherzer', 'max')['key_mlbam'].values[0]
print(f"Max Scherzer's player ID is: {scherzer_id}")

Result:
Gathering player lookup table. This may take a moment.
Max Scherzer's player ID is: 453286


*** Entered Generate Node ***


*** Entered Execute Node ***


Executing:
# Step 2: Get Scherzer's curveball data for first week of August 2020
scherzer_data = statcast_pitcher('2020-08-01', '2020-08-07', player_id=scherzer_id)

# Filter to only curveballs
scherzer_curves = scherzer_data[scherzer_data['pitch_type'] == 'CU']
print(f"Retrieved {len(scherzer_curves)} Scherzer curveballs from 2020-08-01 to 

  0%|          | 0/7 [00:00<?, ?it/s]


*** Entered Execute Node ***


Executing:
# Step 4: Get pitching data for all pitchers in first week of August 2020
data = statcast('2020-08-01', '2020-08-07')
print(f"Retrieved {len(data)} pitches from 2020-08-01 to 2020-08-07")


100%|██████████| 7/7 [00:13<00:00,  1.92s/it]



Result:
This is a large query, it may take a moment to complete
Retrieved 24877 pitches from 2020-08-01 to 2020-08-07


*** Entered Generate Node ***


*** Entered Execute Node ***


Executing:
# Step 5: Filter to only curveballs 
curves = data[data['pitch_type'] == 'CU']

# Step 6: Group by pitcher and calculate average metrics
avg_curves = curves.groupby('pitcher').agg({'release_speed': 'mean',
                                            'release_spin_rate': 'mean',
                                            'pfx_x': 'mean',
                                            'pfx_z': 'mean'}).reset_index()

print(f"Calculated average curveball metrics for {len(avg_curves)} pitchers")

Result:
Calculated average curveball metrics for 215 pitchers


*** Entered Generate Node ***


*** Entered Execute Node ***


Executing:
# Step 7: Combine Scherzer's metrics with all pitcher metrics  
scherzer_avg_df = pd.DataFrame({'pitcher': scherzer_id,
                                'release_speed': sc

In [9]:
task = 'yes'
result = baseball_lambda.execute_workflow(task, session_id)
print(result)


*** Entered Orchestrate Node ***

*** Entered Memorize Node ***

Thank you, task has been commited to memory
